In [0]:
# =============================================================================
# CAMADA SILVER - CARDPRICES - MAGIC: THE GATHERING
# =============================================================================
"""
Script Python para processamento da tabela TB_FATO_SILVER_CARDPRICES
Transformação e limpeza de dados da Bronze para Silver

USO DE SILVER_UTILS.PY:
- Centralização de funções comuns
- Padronização de processamento
- Redução de código duplicado
"""

# =============================================================================
# BIBLIOTECAS UTILIZADAS
# =============================================================================
import logging
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *

# =============================================================================
# CARREGAMENTO DO MÓDULO UTILITÁRIO
# =============================================================================
# Importar funções do silver_utils usando %run (Databricks)
%run ./silver_utils

# =============================================================================
# CONFIGURAÇÃO INICIAL
# =============================================================================

def get_secret(secret_name, default_value=None):
    try:
        return dbutils.secrets.get(scope="mtg-pipeline", key=secret_name)
    except:
        if default_value is not None:
            print(f"Segredo '{secret_name}' não encontrado, usando valor padrão")
            return default_value
        else:
            print(f"Segredo obrigatório '{secret_name}' não encontrado")
            raise Exception(f"Segredo '{secret_name}' não configurado")

def setup_logging():
    """Configura logging para o script"""
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s'
    )
    return logging.getLogger(__name__)

def transform_cardprices_silver(df):
    """
    Transformação específica para tabela CardPrices
    Inclui lógica específica para CardPrices além das transformações padrão
    """
    if not df:
        return None
    
    logger = logging.getLogger(__name__)
    logger.info("Iniciando transformações específicas para CardPrices...")
    
    # Filtro temporal (últimos 5 anos)
    df = apply_temporal_filter(df, months_back=60)
    
    # Padronização de nomes (Title Case)
    df = apply_standard_cleaning(
        df,
        name_columns=["NME_CARD", "NME_RARITY"],
        numeric_columns=["VLR_USD", "VLR_EUR", "VLR_TIX", "VLR_MARKET", "VLR_LOW", "VLR_HIGH"]
    )
    
    # Conversão de COD_SET para maiúsculas
    df = df.withColumn("COD_SET", upper(col("COD_SET")))
    
    # Limpeza e tratamento de nulos - colunas VLR_ como FLOAT com 2 casas decimais
    for colname in ["VLR_USD", "VLR_EUR", "VLR_TIX", "VLR_MARKET", "VLR_LOW", "VLR_HIGH"]:
        if colname in df.columns:
            df = df.withColumn(colname, round(coalesce(col(colname).cast("float"), lit(0.0)), 2))
    
    # Conversão de datas
    df = df.withColumn("DT_INGESTION", col("DT_INGESTION").cast("date"))
    
    # Adicionar colunas de particionamento
    df = add_partition_columns(df, "RELEASE_YEAR", "RELEASE_MONTH")
    
    # Seleção final de colunas
    colunas_finais = [
        "ID_CARD", "NME_CARD", "VLR_USD", "VLR_EUR", "VLR_TIX",
        "DESC_PRICES", "VLR_MARKET", "VLR_LOW",
        "VLR_HIGH", "NME_PRICE_STATUS", "NME_RARITY", "COD_SET",
        "DT_INGESTION", "ANO_PART", "MES_PART"
    ]
    
    # Filtrar colunas que existem no DataFrame
    colunas_disponiveis = [c for c in colunas_finais if c in df.columns]
    df_final = df.select(*colunas_disponiveis)
    
    logger.info(f"Transformação CardPrices concluída: {df_final.count()} registros")
    return df_final

# =============================================================================
# CONFIGURAÇÃO
# =============================================================================

# Configuração manual
config = create_manual_config("magic_the_gathering", get_secret("s3_bucket"))

# Setup Unity Catalog
setup_unity_catalog(config['catalog_name'], config['schema_silver'])



In [0]:
# =============================================================================
# PROCESSAMENTO USANDO SILVER_UTILS
# =============================================================================
# Criar processor
processor = SilverTableProcessor("TB_FATO_SILVER_CARDPRICES", config)

# Extração da Bronze
df_bronze = processor.extract_from_bronze("TB_BRONZE_CARDPRICES")

# Aplicar transformação específica
df_silver = processor.transform_data(df_bronze, transform_cardprices_silver)

# Salvar na Silver com particionamento
processor.save_silver_table(
    df_silver, 
    partition_cols=["ANO_PART", "MES_PART"]
)

# =============================================================================
# VALIDAÇÃO E LOGS
# =============================================================================
if df_silver:
    print(f"Processamento concluído com sucesso!")
    print(f"Registros processados: {df_silver.count()}")
    print(f"Colunas finais: {df_silver.columns}")
else:
    print("Falha no processamento - DataFrame vazio") 